In [128]:
import pandas as pd
import datetime
data1=pd.read_csv('csv/dane_antropo_13.csv')
data2=pd.read_csv('csv/dane_inbody_13.csv')

In [129]:
def parse_date_antropo(x):
    date_time = ""
    try:
        #YYYY-MM-DD
        date_time = datetime.datetime.strptime(x, '%Y-%m-%d')
    except ValueError:
        try:
            #DD-MM-YYYY
            date_time = datetime.datetime.strptime(x, '%d-%m-%Y')
        except ValueError:
            try:
                #YYYY.MM.DD
                date_time = datetime.datetime.strptime(x, '%Y.%m.%d')
            except ValueError:
                #DD.MM.YYYY
                date_time = datetime.datetime.strptime(x, '%d.%m.%Y')
    return date_time
def get_column_suffix(date_parsed):
    if date_parsed.year == 2015:
        return 0
    for i in range(2016,2020):
        if date_parsed.year == i and datetime.datetime.fromisoformat(str(i)+'-07-04') < date_parsed:
            return ((i-2015)*2)
        elif date_parsed.year == i:
            return ((i-2015)*2)-1

In [130]:
#usunięcie duplikatów + preparacje
data1.pop('data.badania')
data2.pop('data.badania')
data2.insert(3,'Sex',data2.pop('Sex'))
data2.insert(4,'DataBadania_inbody',data2.pop('DataBadania'))
data2.insert(4,'Id',data2.pop('Id'))

#tworzenie kolumn
out = pd.DataFrame()
column_names = data1.columns.values.tolist()[1:]+data2.columns.values.tolist()[3:]
#najpierw unikaty
out['id'] = ''
for name in data2.columns.values.tolist()[1:3]:
    out[name] = ''
#później powtarzalne
for name in column_names:
    out[name+"_jesien2015"] = ''
for i in range(2016,2020):
    for name in column_names:
        out[name+"_wiosna"+str(i)] = ''
    for name in column_names:
        out[name+"_jesien"+str(i)] = ''

for i in range(0,len(data1)):
    x = data1.loc[i]
    pdate = parse_date_antropo(x['DataBadania'])
    #fix daty
    x.at['DataBadania'] = pdate.strftime("%Y-%m-%d")
    #dopasuj nazwy kolumn
    n = get_column_suffix(pdate)
    ind = out.columns[3+(n*20)+(218*n):(n*20)+23+(218*n)]
    ind = ind.insert(0,'id')
    x.index = ind
    #dodaj nowy wiersz lub przyklej do istniejącego
    if (out['id'] == x['id']).any():
        out = pd.concat([out, x.to_frame().T])
        out = out.groupby('id',as_index=False,sort=False).last()
    else:
        out = out.append(x)

for i in range(0,len(data2)):
    x = data2.loc[i]
    pdate = datetime.datetime.strptime(x['DataBadania_inbody'], '%Y.%m.%d. %H:%M:%S')
    #dopasuj nazwy kolumn
    n = get_column_suffix(pdate)
    ind = out.columns[3+(20*(n+1))+(n*218):221+(20*(n+1))+(n*218)]
    ind = ind.insert(0,'id')
    ind = ind.insert(1,'DateOfBirth')
    ind = ind.insert(2,'Sex')
    x.index = ind
    #dodaj nowy wiersz lub przyklej do istniejącego
    if (out['id'] == x['id']).any():
        out = pd.concat([out, x.to_frame().T])
        out = out.groupby('id',as_index=False,sort=False).last()
    else:
        out = out.append(x)
        
#v2, po dodaniu wartości tylko zreindeksuj kolumny by fajnie wyglądało :P
reindex_columns = ["id"]
for name in data2.columns.values.tolist()[1:3]:
    reindex_columns += [name]
for name in column_names:
    reindex_columns += [name+"_jesien2015"]
    for i in range(2016,2020):
        reindex_columns += [name+"_wiosna"+str(i)]
        reindex_columns += [name+"_jesien"+str(i)]
out = out.reindex(reindex_columns,axis=1)

out = out.sort_values(by='id')
out.to_csv('out.csv')